## Getting Started with SpanMarker
[SpanMarker](https://github.com/tomaarsen/SpanMarkerNER) is an accessible yet powerful Python module for training Named Entity Recognition models.

In this notebook, we'll have a look at how to train an NER model using SpanMarker.

### Setup
First of all, the `span_marker` Python module needs to be installed. If we want to use [Weights and Biases](https://wandb.ai/) for logging, we can install `span_marker` using the `[wandb]` extra.

In [1]:
%pip install span_marker
# %pip install span_marker[wandb]

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.9/50.9 kB 2.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 280.0/280.0 kB 12.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 510.5/510.5 kB 36.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 12.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.6/43.6 kB 6.6 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 15.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 17.1 MB/s eta 0:00:00
  Created wheel for seqeval: filename=seqeval-1.2.2-py3-none-any.whl size=16162 sha256=5cc8bf5648137961fa0bf9812682680163af7c52e14e8f7fe9bc02d157e4b66b
  Stored in directory: /root/.cache/pip/wheels/1a/67/4a/ad4082dd7dfc30f2abfe4d80a2ed5926a506eb8a972b4767fa
Successfully built seqeval


### Loading the dataset
For this example, we'll load the challenging [FewNERD supervised dataset](https://huggingface.co/datasets/DFKI-SLT/few-nerd) from the Hugging Face hub using 🤗 Datasets.

In [2]:
from datasets import load_dataset

dataset_id = "DFKI-SLT/few-nerd"
dataset = load_dataset(dataset_id, "supervised")
dataset

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:88: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/131767 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/18824 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/37648 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 131767
    })
    validation: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 18824
    })
    test: Dataset({
        features: ['id', 'tokens', 'ner_tags', 'fine_ner_tags'],
        num_rows: 37648
    })
})

Let's inspect some samples to get a feel for the data.

In [ ]:
for sample in dataset["train"].select(range(3)):
    print(sample)

{'id': '0', 'tokens': ['Paul', 'International', 'airport', '.'], 'ner_tags': [0, 0, 0, 0], 'fine_ner_tags': [0, 0, 0, 0]}
{'id': '1', 'tokens': ['It', 'starred', 'Hicks', "'s", 'wife', ',', 'Ellaline', 'Terriss', 'and', 'Edmund', 'Payne', '.'], 'ner_tags': [0, 0, 7, 0, 0, 0, 7, 7, 0, 7, 7, 0], 'fine_ner_tags': [0, 0, 51, 0, 0, 0, 50, 50, 0, 50, 50, 0]}
{'id': '2', 'tokens': ['``', 'Time', '``', 'magazine', 'said', 'the', 'film', 'was', '``', 'a', 'multimillion', 'dollar', 'improvisation', 'that', 'does', 'everything', 'but', 'what', 'the', 'title', 'promises', "''", 'and', 'suggested', 'that', '``', 'writer', 'George', 'Axelrod', '(', '``', 'The', 'Seven', 'Year', 'Itch', '``', ')', 'and', 'director', 'Richard', 'Quine', 'should', 'have', 'taken', 'a', 'hint', 'from', 'Holden', '[', "'s", 'character', 'Richard', 'Benson', ']', ',', 'who', 'writes', 'his', 'movie', ',', 'takes', 'a', 'long', 'sober', 'look', 'at', 'what', 'he', 'has', 'wrought', ',', 'and', 'burns', 'it', '.', "''"], 'n

As you can see, this dataset contains `tokens`, `ner_tags` and a `fine_ner_tags` columns. Let's have a look at which labels these last two represent using the [Dataset features](https://huggingface.co/docs/datasets/about_dataset_features).

In [3]:
labels = dataset["train"].features["ner_tags"].feature.names
print(labels)

['O', 'art', 'building', 'event', 'location', 'organization', 'other', 'person', 'product']


In [4]:
fine_labels = dataset["train"].features["fine_ner_tags"].feature.names
print(fine_labels)

['O', 'art-broadcastprogram', 'art-film', 'art-music', 'art-other', 'art-painting', 'art-writtenart', 'building-airport', 'building-hospital', 'building-hotel', 'building-library', 'building-other', 'building-restaurant', 'building-sportsfacility', 'building-theater', 'event-attack/battle/war/militaryconflict', 'event-disaster', 'event-election', 'event-other', 'event-protest', 'event-sportsevent', 'location-GPE', 'location-bodiesofwater', 'location-island', 'location-mountain', 'location-other', 'location-park', 'location-road/railway/highway/transit', 'organization-company', 'organization-education', 'organization-government/governmentagency', 'organization-media/newspaper', 'organization-other', 'organization-politicalparty', 'organization-religion', 'organization-showorganization', 'organization-sportsleague', 'organization-sportsteam', 'other-astronomything', 'other-award', 'other-biologything', 'other-chemicalthing', 'other-currency', 'other-disease', 'other-educationaldegree', '

For the purposes of this tutorial, let's stick with the `ner_tags` coarse-grained labels, but I challenge you to modify this Notebook to train for the fine labels. For the SpanMarker model, any dataset can be used as long as it has a `tokens` and a `ner_tags` column. The `ner_tags` can be annotated using the IOB, IOB2, BIOES or BILOU labeling scheme, but also regular unschemed labels like in this FewNERD example can be used.

### Initializing a `SpanMarkerModel`
A SpanMarker model is initialized via [SpanMarkerModel.from_pretrained](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.modeling.html#span_marker.modeling.SpanMarkerModel.from_pretrained). This method will be familiar to those who know 🤗 Transformers. It accepts either a path to a local model or the name of a model on the [Hugging Face Hub](https://huggingface.co/models).

Importantly, the model can *either* be an encoder or an already trained and saved SpanMarker model. As we haven't trained anything yet, we will use an encoder. To learn how to load and use a saved SpanMarker model, please have a look at the [Loading & Inferencing](model_loading.ipynb) notebook.

Reasonable options for encoders include BERT, RoBERTa, mBERT, XLM-RoBERTa, etc., which means that the following are all good options:

* [prajjwal1/bert-tiny](https://huggingface.co/prajjwal1/bert-tiny)
* [prajjwal1/bert-mini](https://huggingface.co/prajjwal1/bert-mini)
* [prajjwal1/bert-small](https://huggingface.co/prajjwal1/bert-small)
* [prajjwal1/bert-medium](https://huggingface.co/prajjwal1/bert-medium)
* [bert-base-cased](https://huggingface.co/bert-base-cased)
* [bert-large-cased](https://huggingface.co/bert-large-cased)
* [bert-base-multilingual-cased](https://huggingface.co/bert-base-multilingual-cased)
* [bert-base-multilingual-uncased](https://huggingface.co/bert-base-multilingual-uncased)
* [roberta-base](https://huggingface.co/roberta-base)
* [roberta-large](https://huggingface.co/roberta-large)
* [xlm-roberta-base](https://huggingface.co/xlm-roberta-base)
* [xlm-roberta-large](https://huggingface.co/xlm-roberta-large)

Not all encoders work though, they **must** allow for `position_ids` as an input argument, which disqualifies DistilBERT, T5, DistilRoBERTa, ALBERT & BART.

Additionally, it's important to consider that cased models typically demand consistent capitalization in the inference data, aligning with how the training data is formatted. In simpler terms, if your training data consistently uses correct capitalization, but your inference data does not, it may lead to suboptimal performance. In such cases, you might find an uncased model more suitable. Although it may exhibit slightly lower F1 scores on the testing set, it remains functional regardless of capitalization, making it potentially more effective in real-world scenarios.

We'll use `"bert-base-cased"` for this notebook. If you're running this on Google Colab, be sure to set hardware accelerator to "GPU" in `Runtime` > `Change runtime type`.

In [5]:
from span_marker import SpanMarkerModel, SpanMarkerModelCardData

encoder_id = "bert-base-cased"
model = SpanMarkerModel.from_pretrained(
    # Required arguments
    encoder_id,
    labels=labels,
    # Optional arguments
    model_max_length=256,
    entity_max_length=8,
    # To improve the generated model card
    model_card_data=SpanMarkerModelCardData(
        language=["en"],
        license="cc-by-sa-4.0",
        encoder_id=encoder_id,
        dataset_id=dataset_id,
    )
)

config.json:   0%|          | 0.00/570 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/436M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/49.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/213k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/436k [00:00<?, ?B/s]

For us, these warnings are expected, as we are initializing `BertModel` for a new task.

Note that we provided `SpanMarkerModel.from_pretrained` with a list of our labels. This is required when training a new model using an encoder. Furthermore, we can specify some useful configuration parameters from `SpanMarkerConfig`, such as:

* `model_max_length`: The maximum number of tokens that the model will process. If you only use short sentences for your model, reducing this number may help training and inference speeds with no loss in performance. Defaults to the encoder maximum, or 512 if the encoder doesn't have a maximum.
* `entity_max_length`: The total number of words that one entity can be. Defaults to 8.
* `model_card_data`: A [SpanMarkerModelCardData](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.model_card.html#span_marker.model_card.SpanMarkerModelCardData) instance where you can provide a lot of useful data about your model. This data will be automatically included in a generated model card whenever a model is saved or pushed to the Hugging Face Hub.
    * Consider adding `language`, `license`, `model_id`, `encoder_id` and `dataset_id` to improve the generated model card README.md file.

### Training
At this point, our model is already ready for training! We can import [TrainingArguments](https://huggingface.co/docs/transformers/main_classes/trainer#transformers.TrainingArguments) directly from 🤗 Transformers as well as the SpanMarker `Trainer`. The `Trainer` is a subclass of the 🤗 Transformers [Trainer](https://huggingface.co/docs/transformers/main_classes/trainer) that simplifies some tasks for you, but otherwise it works just like the regular `Trainer`.

This next snippet shows some reasonable defaults. Feel free to adjust the batch size to a lower value if you experience out of memory exceptions.

In [7]:
from transformers import TrainingArguments

args = TrainingArguments(
    output_dir="models/span-marker-bert-base-fewnerd-coarse-super",
    learning_rate=5e-5,
    gradient_accumulation_steps=2,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=1,
    evaluation_strategy="steps",
    save_strategy="steps",
    eval_steps=200,
    push_to_hub=False,
    logging_steps=50,
    fp16=True,
    warmup_ratio=0.1,
    dataloader_num_workers=4,
)

Now we can create a SpanMarker [Trainer](https://tomaarsen.github.io/SpanMarkerNER/api/span_marker.trainer.html#span_marker.trainer.Trainer) in the same way that you would initialize a 🤗 Transformers `Trainer`. We'll train on a subsection of the data to save us some time. Amazingly, this `Trainer` will automatically create logs using exactly the logging tools that you have installed. With other words, if you prefer logging with [Tensorboard](https://www.tensorflow.org/tensorboard), all that you have to do is install it.

In [8]:
from span_marker import Trainer

trainer = Trainer(
    model=model,
    args=args,
    train_dataset=dataset["train"].select(range(8000)),
    eval_dataset=dataset["validation"].select(range(2000)),
)

INFO:span_marker.label_normalizer:No labeling scheme detected: all label IDs belong to individual entity classes.


Let's start training!

In [ ]:
trainer.train()

Label normalizing the train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

Tokenizing the train dataset:   0%|          | 0/8000 [00:00<?, ? examples/s]

These are the frequencies of the missed entities due to maximum entity length out of 20351 total entities:
- 24 missed entities with 9 words (0.117930%)
- 15 missed entities with 10 words (0.073706%)
- 14 missed entities with 11 words (0.068793%)
- 7 missed entities with 12 words (0.034396%)
- 5 missed entities with 13 words (0.024569%)
- 2 missed entities with 15 words (0.009828%)
- 1 missed entities with 17 words (0.004914%)
- 1 missed entities with 19 words (0.004914%)


Spreading data between multiple samples:   0%|          | 0/8000 [00:00<?, ? examples/s]

INFO:span_marker.trainer:Spread 8000 sentences across 9818 samples, a 22.725000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


...

Tokenizing the evaluation dataset:   0%|          | 0/2000 [00:00<?, ? examples/s]

These are the frequencies of the missed entities due to maximum entity length out of 5203 total entities:
- 5 missed entities with 9 words (0.096098%)
- 5 missed entities with 10 words (0.096098%)
- 2 missed entities with 11 words (0.038439%)
- 1 missed entities with 12 words (0.019220%)
- 3 missed entities with 13 words (0.057659%)


Spreading data between multiple samples:   0%|          | 0/2000 [00:00<?, ? examples/s]

INFO:span_marker.trainer:Spread 2000 sentences across 2449 samples, a 22.450000% increase. You can increase `model_max_length` or `marker_max_length` to decrease the number of samples, but recognize that longer samples are slower.
/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:557: UserWarning: This DataLoader will create 4 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(_create_warning_msg(


Step,Training Loss,Validation Loss,Overall Precision,Overall Recall,Overall F1,Overall Accuracy
200,0.036000,0.032041,0.716430,0.506073,0.593153,0.880421
400,0.026200,0.024405,0.737778,0.704068,0.720529,0.923075
600,0.023700,0.021798,0.776012,0.705996,0.739350,0.927971


And now the final step is to compute the model's performance.

In [ ]:
metrics = trainer.evaluate()
metrics

{'eval_loss': 0.019375888630747795,
 'eval_art': {'precision': 0.7661290322580645,
  'recall': 0.7723577235772358,
  'f1': 0.7692307692307692,
  'number': 246},
 'eval_building': {'precision': 0.5842293906810035,
  'recall': 0.6127819548872181,
  'f1': 0.5981651376146789,
  'number': 266},
 'eval_event': {'precision': 0.5497382198952879,
  'recall': 0.5965909090909091,
  'f1': 0.5722070844686648,
  'number': 176},
 'eval_location': {'precision': 0.8036732108929703,
  'recall': 0.8409542743538767,
  'f1': 0.8218911917098446,
  'number': 1509},
 'eval_organization': {'precision': 0.7474226804123711,
  'recall': 0.6998069498069498,
  'f1': 0.7228315054835494,
  'number': 1036},
 'eval_other': {'precision': 0.6775818639798489,
  'recall': 0.5604166666666667,
  'f1': 0.61345496009122,
  'number': 480},
 'eval_person': {'precision': 0.8636363636363636,
  'recall': 0.9063313096270599,
  'f1': 0.8844688954718578,
  'number': 1153},
 'eval_product': {'precision': 0.7366666666666667,
  'recall':

Additionally, we should evaluate using the test set.

In [ ]:
trainer.evaluate(dataset["test"].select(range(2000)), metric_key_prefix="test")

This SpanMarker model won't be able to predict 0.285605% of all annotated entities in the evaluation dataset. This is caused by the SpanMarkerModel maximum entity length of 8 words.
These are the frequencies of the missed entities due to maximum entity length out of 5252 total entities:
- 5 missed entities with 9 words (0.095202%)
- 1 missed entities with 10 words (0.019040%)
- 3 missed entities with 11 words (0.057121%)
- 2 missed entities with 12 words (0.038081%)
- 1 missed entities with 13 words (0.019040%)
- 1 missed entities with 17 words (0.019040%)
- 1 missed entities with 19 words (0.019040%)
- 1 missed entities with 40 words (0.019040%)


{'test_loss': 0.01918497122824192,
 'test_art': {'precision': 0.7419354838709677,
  'recall': 0.7488372093023256,
  'f1': 0.7453703703703703,
  'number': 215},
 'test_building': {'precision': 0.6236559139784946,
  'recall': 0.710204081632653,
  'f1': 0.6641221374045801,
  'number': 245},
 'test_event': {'precision': 0.6153846153846154,
  'recall': 0.5529953917050692,
  'f1': 0.5825242718446603,
  'number': 217},
 'test_location': {'precision': 0.812192118226601,
  'recall': 0.8515171078114913,
  'f1': 0.8313898518751971,
  'number': 1549},
 'test_organization': {'precision': 0.7320754716981132,
  'recall': 0.6897777777777778,
  'f1': 0.7102974828375286,
  'number': 1125},
 'test_other': {'precision': 0.7375886524822695,
  'recall': 0.6328600405679513,
  'f1': 0.6812227074235807,
  'number': 493},
 'test_person': {'precision': 0.8805309734513275,
  'recall': 0.9061930783242259,
  'f1': 0.8931777378815081,
  'number': 1098},
 'test_product': {'precision': 0.6641221374045801,
  'recall': 

Let's try the model out with some predictions. For this we can use the `model.predict` method, which accepts either:

* A sentence as a string.
* A tokenized sentence as a list of strings.
* A list of sentences as a list of strings.
* A list of tokenized sentences as a list of lists of strings.

The method returns a list of dictionaries for each sentence, with the following keys:

* `"label"`: The string label for the found entity.
* `"score"`: The probability score indicating the model its confidence.
* `"span"`: The entity span as a string.
* `"word_start_index"` and `"word_end_index"`: Integers useful for indexing the entity from a tokenized sentence.
* `"char_start_index"` and `"char_end_index"`: Integers useful for indexing the entity from a string sentence.

In [ ]:
sentences = [
    "The Ninth suffered a serious defeat at the Battle of Camulodunum under Quintus Petillius Cerialis in the rebellion of Boudica (61), when most of the foot-soldiers were killed in a disastrous attempt to relieve the besieged city of Camulodunum (Colchester).",
    "He was born in Wellingborough, Northamptonshire, where he attended Victoria Junior School, Westfield Boys School and Sir Christopher Hatton School.",
    "Nintendo continued to sell the revised Wii model and the Wii Mini alongside the Wii U during the Wii U's first release year.",
    "Dorsa has a Bachelor of Music in Composition from California State University, Northridge in 2001, Master of Music in Harpsichord Performance at Cal State Northridge in 2004, and a Doctor of Musical Arts at the University of Michigan, Ann Arbor in 2008."
]

entities_per_sentence = model.predict(sentences)

for entities in entities_per_sentence:
    for entity in entities:
        print(entity["span"], "=>", entity["label"])
    print()

Battle of Camulodunum => event
Quintus Petillius Cerialis => person
Boudica => location
Camulodunum => location
Colchester => location

Wellingborough => location
Northamptonshire => location
Victoria Junior School => organization
Westfield Boys School => organization
Sir Christopher Hatton School => organization

Nintendo => organization
Wii => product
Wii Mini => product
Wii U => product
Wii U => product

Dorsa => person
Bachelor of Music in Composition => other
California State University => organization
Northridge => location
Master of Music in Harpsichord Performance => other
Cal State Northridge => organization
Doctor of Musical Arts => other
University of Michigan => organization
Ann Arbor => location

Very impressive performance for less than 8 minutes trained! 🎉

Once trained, we can save our new model locally. The saved model also comes with a flashy `README.md` such as [this one](https://huggingface.co/tomaarsen/span-marker-bert-base-uncased-bionlp).

In [ ]:
trainer.save_model("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

Or we can push it to the 🤗 Hub like so.

In [ ]:
trainer.push_to_hub(repo_id="span-marker-bert-base-fewnerd-coarse-super")

If we want to use it again, we can just load it using the checkpoint or using the model name on the Hub. This is how it would be done using a local checkpoint.

In [ ]:
# model = SpanMarkerModel.from_pretrained("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

That was all! As simple as that. If we put it all together into a single script, it looks something like this:
```python
from datasets import load_dataset
from span_marker import SpanMarkerModel, SpanMarkerModelCardData, Trainer
from transformers import TrainingArguments

def main():
    dataset_id = "DFKI-SLT/few-nerd"
    dataset = load_dataset(dataset_id, "supervised")
    labels = dataset["train"].features["ner_tags"].feature.names

    encoder_id = "bert-base-cased"
    model = SpanMarkerModel.from_pretrained(
        # Required arguments
        encoder_id,
        labels=labels,
        # Optional arguments
        model_max_length=256,
        entity_max_length=8,
        # To improve the generated model card
        model_card_data=SpanMarkerModelCardData(
            language=["en"],
            license="cc-by-sa-4.0",
            encoder_id=encoder_id,
            dataset_id=dataset_id,
        )
    )

    args = TrainingArguments(
        output_dir="models/span-marker-bert-base-fewnerd-coarse-super",
        learning_rate=5e-5,
        gradient_accumulation_steps=2,
        per_device_train_batch_size=4,
        per_device_eval_batch_size=4,
        num_train_epochs=1,
        evaluation_strategy="steps",
        save_strategy="steps",
        eval_steps=200,
        push_to_hub=False,
        logging_steps=50,
        fp16=True,
        warmup_ratio=0.1,
        dataloader_num_workers=2,
    )

    trainer = Trainer(
        model=model,
        args=args,
        train_dataset=dataset["train"].select(range(8000)),
        eval_dataset=dataset["validation"].select(range(2000)),
    )
    trainer.train()

    metrics = trainer.evaluate()
    print(metrics)

    trainer.save_model("models/span-marker-bert-base-fewnerd-coarse-super/checkpoint-final")

if __name__ == "__main__":
    main()
```

With `wandb` initialized, you can enjoy their very useful training graphs straight in your browser. It ends up looking something like this.
![image](https://user-images.githubusercontent.com/37621491/235196250-15d595f4-6d72-4625-bde9-f3783484997b.png)
![image](https://user-images.githubusercontent.com/37621491/235196335-6f36a7fb-5274-4ce5-a1f3-1d2ad35b26a4.png)

Furthermore, you can use the `wandb` hyperparameter search functionality using the tutorial from the Hugging Face documentation [here](https://huggingface.co/docs/transformers/hpo_train). This transfers very well to the SpanMarker `Trainer`.